<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Deloitte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
We need to find the genre of the person with the most Oscar wins. In case of a tie (multiple people with the same number of wins),
we need to return the person who comes first alphabetically by their name.



In [8]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName("learning").getOrCreate()

nominee_data = [
    ('Jennifer Lawrence', 'P562566', 'Drama', '1990-08-15', 755),
    ('Jonah Hill', 'P418718', 'Comedy', '1983-12-20', 747),
    ('Anne Hathaway', 'P292630', 'Drama', '1982-11-12', 744),
    ('Jennifer Hudson', 'P454405', 'Drama', '1981-09-12', 742),
    ('Rinko Kikuchi', 'P475244', 'Drama', '1981-01-06', 739)
]

# Sample data for oscar_nominees
oscar_data = [
    (2008, 'actress in a leading role', 'Anne Hathaway', 'Rachel Getting Married', 0, 77),
    (2012, 'actress in a supporting role', 'Anne HathawayLes', 'Mis_rables', 1, 78),
    (2006, 'actress in a supporting role', 'Jennifer Hudson', 'Dreamgirls', 1, 711),
    (2010, 'actress in a leading role', 'Jennifer Lawrence', 'Winters Bone', 1, 717),
    (2012, 'actress in a leading role', 'Jennifer Lawrence', 'Silver Linings Playbook', 1, 718),
    (2011, 'actor in a supporting role', 'Jonah Hill', 'Moneyball', 0, 799),
    (2006, 'actress in a supporting role', 'Rinko Kikuchi', 'Babel', 0, 1253)
]

# Define schema for nominee_information
columns_nominee = ["name", "amg_person_id", "top_genre", "birthday", "id"]

# Define schema for oscar_nominees
columns_oscar = ["year", "category", "nominee", "movie", "winner", "id"]

df_nominee=spark.createDataFrame(nominee_data,columns_nominee)

df_nominee.show()

df_oscar=spark.createDataFrame(oscar_data,columns_oscar)

df_oscar.show()

df_oscar_wins=df_oscar.filter("winner==1").groupBy('nominee').agg(count('winner').alias('total_wins'))

df_oscar_wins.show()

joined_df=df_oscar_wins.join(df_nominee,col('nominee')==col('name'),"inner").orderBy(col('total_wins').desc(),col('name'))

joined_df.show()

resultdf=joined_df.select("name","top_genre").first()


print(f"The genre of the person with the most Oscar wins is {resultdf['top_genre']}.")


+-----------------+-------------+---------+----------+---+
|             name|amg_person_id|top_genre|  birthday| id|
+-----------------+-------------+---------+----------+---+
|Jennifer Lawrence|      P562566|    Drama|1990-08-15|755|
|       Jonah Hill|      P418718|   Comedy|1983-12-20|747|
|    Anne Hathaway|      P292630|    Drama|1982-11-12|744|
|  Jennifer Hudson|      P454405|    Drama|1981-09-12|742|
|    Rinko Kikuchi|      P475244|    Drama|1981-01-06|739|
+-----------------+-------------+---------+----------+---+

+----+--------------------+-----------------+--------------------+------+----+
|year|            category|          nominee|               movie|winner|  id|
+----+--------------------+-----------------+--------------------+------+----+
|2008|actress in a lead...|    Anne Hathaway|Rachel Getting Ma...|     0|  77|
|2012|actress in a supp...| Anne HathawayLes|          Mis_rables|     1|  78|
|2006|actress in a supp...|  Jennifer Hudson|          Dreamgirls|     1|